In [1]:
import pandas as pd

In [2]:
meta_muts = pd.read_pickle('./targ_mut_df.pkl')
display(len(meta_muts), meta_muts.head())

39

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest
229085,DDB_MS56_strain_verification,37 1 1 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683}
468632,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,13 51 0 1,710595,SNP,G→T,P18T (CCT→ACT),fur,private,True,{fur},{b0683},{b0683}
468721,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,17 61 0 1,710436,SNP,G→A,H71Y (CAC→TAC),fur,private,True,{fur},{b0683},{b0683}
468785,GLU_16321-M9-NC_000913_3gb-stationary-37-Gluco...,19 50 0 1,710595,SNP,G→T,P18T (CCT→ACT),fur,private,True,{fur},{b0683},{b0683}
445643,GLU_4721-M9-NC_000913_3gb-stationary-37-Glucos...,3 44 0 1,710489,SNP,G→C,A53G (GCT→GGT),fur,private,True,{fur},{b0683},{b0683}


In [3]:
meta_muts = meta_muts[meta_muts['Sequence Change'] != "Δ411 bp"]  # ignoring large deletion for now

In [4]:
# !!! Temporary; need to reprocess all ALE experiment data for this project with the now removed ERS project experiments which were being duplicated by the newer ROS_eAMP ALE project/experiment.

meta_muts = meta_muts[meta_muts.exp != 'GLU_16321-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_18361-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']
meta_muts = meta_muts[meta_muts.exp != 'GLU_4721-M9-NC_000913_3gb-stationary-37-Glucose2-NH4Cl1-KH2PO43-MgSO4024-O2']

In [5]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.mut import get_mut_size, get_mob_size, is_genetic_mut, is_coding_mut, predict_mutation_effect_on_feature, get_SNP_aa_pos, get_DEL_INS_MOB_aa_start_pos, get_DEL_AA_range


meta_muts["genetic"] = meta_muts["Details"].apply(is_genetic_mut)
meta_muts["coding"] = meta_muts["Details"].apply(is_coding_mut)
meta_muts["mutation size"] = meta_muts.apply(get_mut_size, axis=1)  # get_mut_size currently doesn't return MOB mut sizes. Isn't something currently necessary.

In [6]:
from aledbmutil.mut import get_SNP_aa_pos, get_DEL_AA_range

def _get_SNP_AA_range(mut_details_str):
    SNP_AA_range = ()
    AA_pos = get_SNP_aa_pos(mut_details_str)
    SNP_AA_range = (AA_pos, AA_pos)
    return SNP_AA_range

meta_muts["AA range"] = None
# meta_muts["AA range"] = meta_muts.apply(
#     lambda r: _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) else r["AA range"],
#     axis=1)
meta_muts["AA range"] = meta_muts.apply(lambda r:
                                        _get_SNP_AA_range(r["Details"]) if ((r["Mutation Type"] == "SNP") & (r["coding"]==1)) \
                                        else (get_DEL_AA_range(r["Details"]) if ((r["Mutation Type"] == "DEL") & (r["coding"]==1)) \
                                        else r["AA range"]),
                                        axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range
229085,DDB_MS56_strain_verification,37 1 1 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)"
159670,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,16 28 1 1,710522,SNP,C→T,R42H (CGT→CAT),fur,private,True,{fur},{b0683},{b0683},True,1,"(42, 42)"
189532,MS56,7 48 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)"
189761,MS56,9 44 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)"
188967,MS56,10 42 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)"


In [7]:
# currently only tracking seq changes with SNPs for ALEdb mutations for later processing with AA property change trends
meta_muts['AA ref seq'] = meta_muts.apply(lambda m: m['Details'][0] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts['AA seq change'] = meta_muts.apply(lambda m: m['Details'].split(' (')[0][-1] if m["Mutation Type"]=='SNP' else '', axis=1)
meta_muts.head()

,exp,sample,Position,Mutation Type,Sequence Change,Details,Gene,ALEdb source,genetic,clean genes,bnums,genes of interest,coding,mutation size,AA range,AA ref seq,AA seq change
229085,DDB_MS56_strain_verification,37 1 1 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)",R,S
159670,GLU_ALE4-M9-NC_000913_3gb-stationary-37-Glucos...,16 28 1 1,710522,SNP,C→T,R42H (CGT→CAT),fur,private,True,{fur},{b0683},{b0683},True,1,"(42, 42)",R,H
189532,MS56,7 48 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)",R,S
189761,MS56,9 44 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)",R,S
188967,MS56,10 42 0 1,710439,SNP,G→T,R70S (CGC→AGC),fur,private,True,{fur},{b0683},{b0683},True,1,"(70, 70)",R,S


In [8]:
exp_sel_pres_d = {
    'ROS_eAMP': "paraquat",  # starting strains were also pre-evolved on glucose
 }

meta_muts['selection pressure'] = meta_muts.apply(lambda r: 'unpublished' if r['ALEdb source'] == 'private' else '', axis=1)
meta_muts['selection pressure'] = meta_muts.exp.apply(lambda e: exp_sel_pres_d[e] if e in exp_sel_pres_d.keys() else 'unpublished')
assert('' not in set(meta_muts['selection pressure']))

In [9]:
# TODO: fix warnings with append
import warnings
warnings.filterwarnings('ignore')


aa_var_df = pd.DataFrame()

from aledbmutil.mut import get_mut_size  #, get_DEL_INS_MOB_aa_start_pos


for mt, mut_type_mut_df in meta_muts.groupby("Mutation Type"):
    assert mt != "MOB", 'Currently not handilng MOB. The below INS logic might be able to handle this.'
    if mt == "SNP":
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = ""
            details_str = m["Details"]
            details_str = details_str[0:details_str.find(" (")]
            
            if details_str[-1] == '*':  # Could have them show the effected downstream positions like with ACS SynBio paper. 
                mut_eff = "truncation"
            elif details_str[0] == details_str[-1]:
                mut_eff = "synonymous substitution"
            elif details_str[0] != details_str[-1]:
                mut_eff = "nonsynonymous substitution"
            else:
                mut_eff = "unknown"
            
            aa_pos = int(details_str[1:-1])
            aa_var_df= aa_var_df.append({
                "AA pos": aa_pos,
                "mutation type": "SUB",
                "mutation effect": mut_eff,
                "mutation source": m['selection pressure'],
                'AA ref seq': m['AA ref seq'],
                'AA seq change': m['AA seq change'],
            }, ignore_index=True)
    if (mt == 'DEL'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            mut_eff = "deletion"  # currently assuming only insertions remain
            mut_size = m['mutation size']
            if mut_size % 3 != 0:
                assert mut_size % 3 != 0, 'not dealing with frameshifts yet' # Could have them show the effected downstream positions like with ACS SynBio paper.
            else:
                for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                    aa_var_df= aa_var_df.append({
                        "AA pos": aa_pos,
                        "mutation type": mt,
                        "mutation effect": mut_eff,
                        "mutation source": m['selection pressure'],
                        'AA ref seq': '',
                        'AA seq change': '',
                    },ignore_index=True)
    # if (mt == 'INS'):
    #     for _, m in mut_type_mut_df.iterrows():
    #         mut_eff = "insertion"  # currently assuming only insertions remain
    #         mut_size = get_mut_size(m)
    #         if mut_size % 3 != 0:
    #             mut_eff = "frameshift"
    #         aa_var_df= aa_var_df.append({
    #             "AA pos": get_DEL_INS_MOB_aa_start_pos(m["Details"]),
    #             "mutation type": mt,
    #             "mutation effect": mut_eff,
    #             "mutation source": m['selection pressure'],
    #             'AA ref seq': '',
    #             'AA seq change': '',
    #         },ignore_index=True)
    if (mt == 'SUB'):  # Unravel since downstream analysis just looks at individual mutated AAs no matter if they come from the same allele.
        for _, m in mut_type_mut_df.iterrows():
            for aa_pos in range(m['AA range'][0], m['AA range'][1]+1):
                aa_var_df= aa_var_df.append({
                    "AA pos": aa_pos,
                    "mutation type": mt,
                    "mutation effect": "nonsynonymous substitution",
                    "mutation source": m['selection pressure'],
                    'AA ref seq': '',
                    'AA seq change': '',
                },ignore_index=True)


display(len(aa_var_df), aa_var_df.head())

33

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change
0,70,SUB,nonsynonymous substitution,unpublished,R,S
1,42,SUB,nonsynonymous substitution,unpublished,R,H
2,70,SUB,nonsynonymous substitution,unpublished,R,S
3,70,SUB,nonsynonymous substitution,unpublished,R,S
4,70,SUB,nonsynonymous substitution,unpublished,R,S


In [10]:
aa_var_cnt_df = pd.DataFrame()
gcols = ['AA pos', 'mutation type', 'mutation effect', 'mutation source', 'AA ref seq', 'AA seq change']
for g, gdf in aa_var_df.groupby(gcols):  # Essentially just finding duplicates
    d = dict(zip(gcols, g))
    d['mutation count'] = len(gdf)
    aa_var_cnt_df = aa_var_cnt_df.append(d, ignore_index=True)
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count
0,7,SUB,nonsynonymous substitution,unpublished,A,P,1
1,14,SUB,nonsynonymous substitution,unpublished,K,N,1
2,14,SUB,nonsynonymous substitution,unpublished,K,Q,1
3,18,SUB,nonsynonymous substitution,paraquat,P,T,3
4,18,SUB,nonsynonymous substitution,unpublished,P,T,1


In [11]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import const


aa_var_cnt_df['mutation effect color'] = aa_var_cnt_df['mutation effect'].apply(lambda e: const.MUTATION_TYPE_COLOR_PALETTE[e])
aa_var_cnt_df.head()

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color
0,7,SUB,nonsynonymous substitution,unpublished,A,P,1,#DD8452
1,14,SUB,nonsynonymous substitution,unpublished,K,N,1,#DD8452
2,14,SUB,nonsynonymous substitution,unpublished,K,Q,1,#DD8452
3,18,SUB,nonsynonymous substitution,paraquat,P,T,3,#DD8452
4,18,SUB,nonsynonymous substitution,unpublished,P,T,1,#DD8452


In [12]:
sel_pres_clr_d = {
    # 'FeSO4': '#729ECE',
    # 'Homo sapiens tpi': '#FF9E4A',
    # 'Pyrobaculum aerophilum pgi': '#67BF5C',
    # 'Pyrobaculum aerophilum tpi': '#ED665D',
    # 'Vibrio cholerae tpi': '#AD8BC9',
    # 'isobutanol': '#ED97CA',
    'paraquat': '#6DCCDA',
    'unpublished': '#AD8BC9'
 }

aa_var_cnt_df['selection pressure color'] = aa_var_cnt_df['mutation source'].apply(lambda e: sel_pres_clr_d[e])
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color
0,7,SUB,nonsynonymous substitution,unpublished,A,P,1,#DD8452,#AD8BC9
1,14,SUB,nonsynonymous substitution,unpublished,K,N,1,#DD8452,#AD8BC9
2,14,SUB,nonsynonymous substitution,unpublished,K,Q,1,#DD8452,#AD8BC9
3,18,SUB,nonsynonymous substitution,paraquat,P,T,3,#DD8452,#6DCCDA
4,18,SUB,nonsynonymous substitution,unpublished,P,T,1,#DD8452,#AD8BC9
5,23,SUB,nonsynonymous substitution,unpublished,L,Q,2,#DD8452,#AD8BC9
6,42,SUB,nonsynonymous substitution,paraquat,R,H,1,#DD8452,#6DCCDA
7,42,SUB,nonsynonymous substitution,unpublished,R,H,1,#DD8452,#AD8BC9
8,53,SUB,nonsynonymous substitution,paraquat,A,G,1,#DD8452,#6DCCDA
9,61,SUB,nonsynonymous substitution,unpublished,Q,E,1,#DD8452,#AD8BC9


In [13]:
def _get_variant_name(variant):
    var_name = ''
    if variant['mutation type'] == 'SUB':
        var_name = variant['mutation type'] + " " + variant['AA ref seq'] + str(variant['AA pos']) + variant['AA seq change']
    return var_name


aa_var_cnt_df['name'] = aa_var_cnt_df.apply(lambda r: _get_variant_name(r), axis=1)
aa_var_cnt_df

,AA pos,mutation type,mutation effect,mutation source,AA ref seq,AA seq change,mutation count,mutation effect color,selection pressure color,name
0,7,SUB,nonsynonymous substitution,unpublished,A,P,1,#DD8452,#AD8BC9,SUB A7P
1,14,SUB,nonsynonymous substitution,unpublished,K,N,1,#DD8452,#AD8BC9,SUB K14N
2,14,SUB,nonsynonymous substitution,unpublished,K,Q,1,#DD8452,#AD8BC9,SUB K14Q
3,18,SUB,nonsynonymous substitution,paraquat,P,T,3,#DD8452,#6DCCDA,SUB P18T
4,18,SUB,nonsynonymous substitution,unpublished,P,T,1,#DD8452,#AD8BC9,SUB P18T
5,23,SUB,nonsynonymous substitution,unpublished,L,Q,2,#DD8452,#AD8BC9,SUB L23Q
6,42,SUB,nonsynonymous substitution,paraquat,R,H,1,#DD8452,#6DCCDA,SUB R42H
7,42,SUB,nonsynonymous substitution,unpublished,R,H,1,#DD8452,#AD8BC9,SUB R42H
8,53,SUB,nonsynonymous substitution,paraquat,A,G,1,#DD8452,#6DCCDA,SUB A53G
9,61,SUB,nonsynonymous substitution,unpublished,Q,E,1,#DD8452,#AD8BC9,SUB Q61E


In [14]:
aa_var_cnt_df.to_csv('fur_ale_muts.csv', index=False)

In [15]:
mut_needle_plot_legend_df = pd.DataFrame()
mut_needle_plot_legend_df = aa_var_cnt_df.copy()
mut_needle_plot_legend_df = mut_needle_plot_legend_df[['mutation source', 'selection pressure color', ]]
mut_needle_plot_legend_df.rename(columns={'mutation source': 'name', 'selection pressure color': 'fill'}, inplace=True)
mut_needle_plot_legend_df['edge'] = 'white'
mut_needle_plot_legend_df.drop_duplicates(inplace=True)

df = aa_var_cnt_df.copy()
df = df[['mutation effect', 'mutation effect color']]
df.rename(columns={'mutation effect': 'name', 'mutation effect color': 'edge'}, inplace=True)
df.drop_duplicates(inplace=True)
df['fill'] = 'white'
mut_needle_plot_legend_df = mut_needle_plot_legend_df.append(df, ignore_index=True)
display(mut_needle_plot_legend_df)

mut_needle_plot_legend_df.to_csv('fur_ale_muts_legend.csv', index=False)

,name,fill,edge
0,unpublished,#AD8BC9,white
1,paraquat,#6DCCDA,white
2,nonsynonymous substitution,white,#DD8452


Creating functional feature annotation output

In [16]:
feats_df = pd.read_pickle('./annot_df.pkl')

# filtering specific to fur's features
feats_df["feature"] = feats_df["feature"].apply(lambda s: s.replace("Fur subunit", "subunit"))
feats_df["feature"] = feats_df["feature"].apply(lambda s: s.replace("dimer", "subunit"))  # "dimer interface" and "Fur subunit interface" are the same thing, though coming from different sources"

# filtering out features that aren't useful
feats_to_remove = [
    'Initiator methionine',
    'FERRIC UPTAKE REGULATION PROTEIN',
    'ZINC UPTAKE REGULATION PROTEIN',
    'Ferric-uptake regulator',
    'Ferric-uptake regulator family',
    'Winged helix-like DNA-binding domain superfamily/Winged helix DNA-binding domain',
    'Winged helix-like DNA-binding domain superfamily',
    'Winged helix DNA-binding domain superfamily',
    'G3DSA:3.30.1490.190',
    'Ferric uptake regulator family',
    'Ferric uptake regulator(Fur) and related metalloregulatory proteins; typically iron-dependent, DNA-binding repressors and activators',
    'Winged helix DNA-binding domain',
    'Ferric-uptake regulator, C-terminal domain',
    'Metal binding',
    # 'Chain',
]
feats_df = feats_df[~(feats_df['feature'].isin(feats_to_remove))]
feats_df.reset_index(inplace=True, drop=True)
feats_df

,feature,start,end,range
0,Dimerization region,85,148,"(85, 64)"
1,Chain,2,148,"(2, 147)"
2,Fe binding site,125,125,"(125, 1)"
3,Fe binding site,108,108,"(108, 1)"
4,Zn binding site,101,101,"(101, 1)"
5,Zn binding site,96,96,"(96, 1)"
6,Zn binding site,93,93,"(93, 1)"
7,Zn binding site,90,90,"(90, 1)"
8,Fe binding site,89,89,"(89, 1)"
9,Fe binding site,87,87,"(87, 1)"


In [17]:
layer = 0
layer_d = dict()
for f in feats_df.feature.unique():
    layer_d[f] = layer
    layer += 1
feats_df['layer'] = feats_df['feature'].apply(lambda f: layer_d[f])
feats_df.head()

,feature,start,end,range,layer
0,Dimerization region,85,148,"(85, 64)",0
1,Chain,2,148,"(2, 147)",1
2,Fe binding site,125,125,"(125, 1)",2
3,Fe binding site,108,108,"(108, 1)",2
4,Zn binding site,101,101,"(101, 1)",3


In [18]:
feat_color_d = dict()
muted=["#4878D0", "#EE854A", "#6ACC64", "#D65F5F", "#956CB4", "#8C613C", "#DC7EC0", "#797979", "#D5BB67", "#82C6E2"]  # from seaborn
feats = list(feats_df.feature.unique())
for i in range(0, len(feats_df.feature.unique())):
    feat_color_d[feats[i]] = muted[i]
feat_color_d

feats_df['color'] = feats_df['feature'].apply(lambda f: feat_color_d[f])

feats_df.drop(columns=["range"], inplace=True)
feats_df.drop_duplicates(inplace=True)

display(feats_df, feat_color_d)
feats_df.to_csv('./fur_feats.csv')

,feature,start,end,layer,color
0,Dimerization region,85,148,0,#4878D0
1,Chain,2,148,1,#EE854A
2,Fe binding site,125,125,2,#6ACC64
3,Fe binding site,108,108,2,#6ACC64
4,Zn binding site,101,101,3,#D65F5F
5,Zn binding site,96,96,3,#D65F5F
6,Zn binding site,93,93,3,#D65F5F
7,Zn binding site,90,90,3,#D65F5F
8,Fe binding site,89,89,2,#6ACC64
9,Fe binding site,87,87,2,#6ACC64


{'Dimerization region': '#4878D0',
 'Chain': '#EE854A',
 'Fe binding site': '#6ACC64',
 'Zn binding site': '#D65F5F',
 'DNA binding region': '#956CB4',
 'subunit interface': '#8C613C',
 'putative DNA binding helix': '#DC7EC0',
 'Cd binding site': '#797979',
 'Cl binding site': '#D5BB67',
 'glycerol binding site': '#82C6E2'}

In [19]:
# Filtering out some features that are not useful for the NGL visual 3D analysis
ngl_feat_df = feats_df.copy()
ngl_feat_df = ngl_feat_df[ngl_feat_df['feature'].isin(['subunit interface', 'DNA binding region', 'Dimerization region'])]

In [20]:
# for NGL
ngl_feat_df["size"] = ngl_feat_df.apply(lambda r: r["end"] - r["start"] + 1, axis=1)
ngl_feat_df = ngl_feat_df[["feature", "start", "end", "color", "size"]]
ngl_feat_df = ngl_feat_df.sort_values("size", ascending=False)
ngl_feat_df = ngl_feat_df.reset_index(drop=True)  # json explicitly uses the indexes

with open('feats.json', 'w') as outfile:   # for NGL
    outfile.write(ngl_feat_df.to_json())
ngl_feat_df

,feature,start,end,color,size
0,DNA binding region,2,84,#956CB4,83
1,Dimerization region,85,148,#4878D0,64
2,subunit interface,124,132,#8C613C,9
3,subunit interface,72,76,#8C613C,5
4,subunit interface,92,94,#8C613C,3
5,subunit interface,120,122,#8C613C,3
6,subunit interface,63,64,#8C613C,2
7,subunit interface,56,57,#8C613C,2
8,subunit interface,52,53,#8C613C,2
9,subunit interface,110,111,#8C613C,2
